In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


from google.colab import drive
drive.mount('/content/drive')

md = '/content/drive/My Drive/Colab Notebooks/Novosti/data/prep_dataset.csv'

Mounted at /content/drive


In [2]:
# 1. Загрузка данных
data = pd.read_csv(md)  # Замените на путь к вашему файлу
texts = data['main_text'].values
labels = data['category'].values

# 2. Предобработка данных
# Преобразование меток в числовой формат
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

X_train_val, X_test, y_train_val, y_test = train_test_split(texts, labels_encoded, test_size=0.2, stratify=labels_encoded, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

In [3]:
# 3. Векторизация текста (для RNN и CNN) и EarlyStopping
max_length = 100  # Максимальная длина последовательности
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_length)
X_val_padded = pad_sequences(X_val_seq, maxlen=max_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length)

y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_val_one_hot = to_categorical(y_val, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

In [6]:
early_stopping = EarlyStopping(
    monitor='val_loss', # Отслеживаем значение val_loss (потери на валидационной выборке)
    patience=2,          # Сколько эпох ждать улучшения, прежде чем остановить обучение
    restore_best_weights=True # Восстанавливаем веса модели, соответствующие лучшей эпохе
)

In [4]:
# 4. Создание моделей

# Модель RNN (LSTM)
def create_rnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
    model.add(LSTM(64))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Модель CNN
def create_cnn_model():
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Модель BERT
def create_bert_model():
    bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
    input_ids = tf.keras.Input(shape=(max_length,), dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(max_length,), dtype=tf.int32)
    embeddings = bert_model(input_ids, attention_mask=attention_mask)[0]
    x = tf.keras.layers.GlobalAveragePooling1D()(embeddings)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
# 5. Обучение моделей и оценка точности

# Обучение RNN
rnn_model = create_rnn_model()
history_rnn = rnn_model.fit(
    X_train_padded, y_train_one_hot,
    validation_data=(X_val_padded, y_val_one_hot),
    epochs=10, # Максимальное количество эпох (EarlyStopping может остановить раньше)
    batch_size=32,
    callbacks=[early_stopping] # Передаем EarlyStopping в callbacks
)
#rnn_model.fit(X_train_padded, y_train_one_hot, epochs=5, batch_size=32, validation_split=0.1)
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test_padded, y_test_one_hot)


Epoch 1/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.5499 - loss: 1.3026 - val_accuracy: 0.7896 - val_loss: 0.7203
Epoch 2/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - accuracy: 0.8534 - loss: 0.5093 - val_accuracy: 0.8116 - val_loss: 0.6731
Epoch 3/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - accuracy: 0.9038 - loss: 0.3322 - val_accuracy: 0.8130 - val_loss: 0.6628
Epoch 4/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.9289 - loss: 0.2409 - val_accuracy: 0.8120 - val_loss: 0.6899
Epoch 5/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 29s 18ms/step - accuracy: 0.9421 - loss: 0.1895 - val_accuracy: 0.8128 - val_loss: 0.7689
543/543 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8164 - loss: 0.6381


In [8]:
# Обучение CNN

cnn_model = create_cnn_model()
history_cnn = cnn_model.fit(
    X_train_padded, y_train_one_hot,
    validation_data=(X_val_padded, y_val_one_hot),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_padded, y_test_one_hot)

Epoch 1/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - accuracy: 0.5804 - loss: 1.2449 - val_accuracy: 0.8230 - val_loss: 0.5501
Epoch 2/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.8955 - loss: 0.3399 - val_accuracy: 0.8326 - val_loss: 0.5280
Epoch 3/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.9536 - loss: 0.1583 - val_accuracy: 0.8203 - val_loss: 0.5586
Epoch 4/10
1628/1628 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9659 - loss: 0.1142 - val_accuracy: 0.8267 - val_loss: 0.6139
543/543 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8354 - loss: 0.5271


In [9]:
# 6. Вывод результатов
print(f"Точность RNN: {rnn_accuracy:.4f}")
print(f"Точность CNN: {cnn_accuracy:.4f}")
#print(f"Точность BERT: {bert_accuracy:.4f}")

Точность RNN: 0.8157
Точность CNN: 0.8334


In [ ]:
# Подготовка данных для BERT
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
X_train_bert = tokenizer_bert(X_train.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='tf')
X_test_bert = tokenizer_bert(X_test.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
max_length = 100  # Убедитесь, что это значение соответствует вашей задаче
num_classes = 8  # Замените на количество классов вашей задачи

In [ ]:
def create_bert_model():

    bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

    input_ids = tf.keras.Input(shape=(max_length,), dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(max_length,), dtype=tf.int32)

    # Используем Lambda слой для преобразования KerasTensor в TensorFlow Tensor
    embeddings = tf.keras.layers.Lambda(lambda x: bert_model(x[0], attention_mask=x[1])[0])([input_ids, attention_mask])

    x = tf.keras.layers.GlobalAveragePooling1D()(embeddings)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Обучение BERT
bert_model = create_bert_model()
bert_model.fit([X_train_bert['input_ids'], X_train_bert['attention_mask']], y_train_one_hot,
                epochs=5, batch_size=32, validation_split=0.1)
bert_loss, bert_accuracy = bert_model.evaluate([X_test_bert['input_ids'], X_test_bert['attention_mask']], y_test_one_hot)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=(['<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=keras_tensor_25>', '<KerasTensor shape=(None, 100), dtype=int32, sparse=False, name=keras_tensor_26>'],)
  • kwargs={'mask': ['None', 'None']}